In [13]:
import pandas as pd

In [14]:
from bs4 import BeautifulSoup

with open("chinese_stations.html", encoding="utf8") as file:
    soup = BeautifulSoup(file, 'html5lib')
    cn_links = soup.find_all('a', href=True)

In [15]:
stations = []
for link in cn_links:
    stations.append(link['href'])

In [16]:
import pickle
with open('aqi.pkl', 'rb') as handle:
    df_data = pickle.load(handle)
with open('cities_done.pkl', 'rb') as handle:
    done = pickle.load(handle)
with open('cities_wait.pkl', 'rb') as handle:
    wait = pickle.load(handle)

In [18]:
import requests as rq
import time
from shapely.geometry import Point
import geopandas as gpd

for station in stations:
    
    if station in done or station in wait:
        continue

    with open('cities_done.pkl', 'rb') as handle:
        done = pickle.load(handle)
    with open('cities_wait.pkl', 'rb') as handle:
        wait = pickle.load(handle)
        
    print(station)
        
    resp = rq.request("GET", station)
    
    if resp.status_code != 200: 
        print("STATUS_CODE ERROR : ", resp.status_code, resp.text)
        continue
    
    soup = BeautifulSoup(resp.content, 'html5lib')
    
    info = soup.find(id = 'aqiwgtinfo')

    if info is None or info.get_text() == "no data":
        wait.append(station)
        with open('cities_wait.pkl', 'wb') as handle:
            pickle.dump(wait, handle)
        continue
    
    try:
        pm25 = soup.find(id = 'cur_pm25').get_text()
    except AttributeError:
        pm25 = None
    
    try:
        co = soup.find(id = 'cur_co').get_text() 
    except AttributeError:
        co = None
    
    try:
        so2 = soup.find(id = 'cur_so2').get_text()
    except AttributeError:
        so2 = None
        
    try:
        o3 = soup.find(id = 'cur_o3').get_text()
    except AttributeError:
        o3 = None
        
    try:
        no2 = soup.find(id = 'cur_no2').get_text()
    except AttributeError:
        no2 = None
        
    try:
        pm10 = soup.find(id = 'cur_pm10').get_text()
    except AttributeError:
        pm10 = None
    
    try:
        temp = soup.find(id = 'cur_t').get_text()
    except AttributeError:
        temp = None
        
    name = soup.find(id = 'aqiwgttitle2').get_text().split("Real-time")[0].strip()
    aqi = soup.find(id = 'aqiwgtvalue').get_text()   

    url = "https://api.waqi.info/search/"
    qs = {"token":"8b556369620cbd18f0fc03b37e3959bd970f0706","keyword": name}
    resp_json = rq.request("GET", url, params=qs)

    if resp_json.status_code != 200: 
        print("STATUS_CODE ERROR : ", resp_json.status_code, resp_json.text)
        continue
        
    json = resp_json.json()
    
    if json['data'] == []:
        wait.append(station)
        with open('cities_wait.pkl', 'wb') as handle:
            pickle.dump(wait, handle)
        continue
    
    coord = json['data'][0]['station']['geo']

    geometry = Point(coord[0], coord[1])
    
    city = station.split("/")[-3]
    
    df_data.loc[len(df_data)] = [city, name, aqi, pm25, pm10, no2, so2, o3, co, temp, coord[0], coord[1], geometry]
    
    done.append(station)
    
    df_data.to_pickle('aqi.pkl')
    with open('cities_done.pkl', 'wb') as handle:
        pickle.dump(done, handle)

In [ ]:
# df_data = pd.DataFrame(columns=('city','name','aqi','pm25','pm10','no2','so2','o3','co','temp','lat','long','geometry'))
# wait = []
#done = []
#with open('cities_wait.pkl', 'wb') as handle:
#    pickle.dump(wait, handle)
#with open('cities_done.pkl', 'wb') as handle:
#    pickle.dump(done, handle)
#df_data.to_pickle('aqi.pkl')